In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch
import torch.autograd
from torch.autograd import Variable
import numpy as np
from scipy.io import loadmat
from scipy.special import sph_harm
import math
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from time import time
from mpl_toolkits.mplot3d import axes3d, Axes3D 
from torch.autograd.gradcheck import zero_gradients

In [3]:
# torch.device("cuda")
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [4]:
no_of_ver = 53215

In [5]:
fileName='Dataset/Coarse_Dataset/Exp_Pca.bin'
with open(fileName, mode='rb') as file: # b is important -> binary
#     fileContent = file.read()
    dim_exp = np.fromfile(file, dtype=np.int32, count=1)
    mu_exp = np.zeros(no_of_ver*3)
    base_exp = np.zeros((no_of_ver*3*dim_exp[0]), dtype=float)
    mu_exp = np.fromfile(file, dtype=np.float32, count=3*no_of_ver)
    base_exp = np.fromfile(file, dtype=np.float32, count=3*no_of_ver*dim_exp[0])

In [6]:
A_exp = torch.tensor(np.array(np.resize(base_exp, (no_of_ver*3, dim_exp[0]))), dtype=torch.float32, requires_grad = True)

In [7]:
data = np.loadtxt('Dataset/Coarse_Dataset/std_exp.txt', delimiter=' ')
data = torch.tensor(data[:,np.newaxis], dtype = torch.float32,requires_grad =True )

In [8]:
temp = loadmat('Dataset/3DDFA_Release/Matlab/ModelGeneration/model_info.mat')
temp['tri'].dtype = np.int16
trimIndex = np.array(temp['trimIndex'][:,0], dtype=np.int32)
trim_ind = np.reshape(np.array([3*trimIndex-2,3*trimIndex-1,3*trimIndex])-1,(no_of_ver*3,),'F')
tri_mesh_data = temp['tri'].T#torch.tensor(temp['tri'].T, dtype=torch.long)

In [9]:
def load(path):
    """takes as input the path to a .pts and returns a list of 
    tuples of floats containing the points in in the form:
    [(x_0, y_0, z_0),
     (x_1, y_1, z_1),
     ...
     (x_n, y_n, z_n)]"""
    with open(path) as f:
        rows = [rows.strip() for rows in f]

    """Use the curly braces to find the start and end of the point data""" 
    head = rows.index('{') + 1
    tail = rows.index('}')

    """Select the point data split into coordinates"""
    raw_points = rows[head:tail]
    coords_set = [point.split() for point in raw_points]

    """Convert entries from lists of strings to tuples of floats"""
    points = [tuple([float(point) for point in coords]) for coords in coords_set]
    return points

In [10]:
lmks_2d = torch.tensor(load('Dataset/300W-Convert/300W-Original/afw/134212_1.pts'))
lmks_3d_ind = np.array(temp['keypoints'], dtype=np.int32)

In [11]:
print("PyTorch version: ", torch.__version__ )
print("CUDA available: ", torch.cuda.is_available())
print("CUDA version: ", torch.version.cuda)

PyTorch version:  1.1.0
CUDA available:  True
CUDA version:  9.0.176


In [12]:
morph_model = loadmat('Dataset/PublicMM1/01_MorphableModel.mat')

In [13]:
shapePCA = morph_model['shapePC']
shapeMU = morph_model['shapeMU']
shapeSTD = morph_model['shapeEV']

texPCA = morph_model['texPC']
texMU = morph_model['texMU']
texSTD = morph_model['texEV']

In [14]:
p_mu = torch.tensor(shapeMU[trim_ind])
b_mu = torch.tensor(texMU[trim_ind])
A_alb = torch.tensor(texPCA[trim_ind,:100])
A_id = torch.tensor(shapePCA[trim_ind,:100])
std_id = torch.tensor(shapeSTD[:100])
std_alb = torch.tensor(texSTD[:100])
std_exp = data

In [15]:
def P(l, m, x):
    pmm = 1.0
    if m>0:
        somx2 = torch.sqrt((1.0-x)*(1.0+x))
        fact = 1.0
        for i in torch.arange(m):
            pmm = -fact*pmm*somx2
            fact = fact+2.0
    if l==m :
        return pmm
    pmmp1 = x*(2.0*m+1.0)*pmm
    if (l==m+1):
        return pmmp1
    pll = 0.0
    for ll in torch.arange(m+2, l+1):
        pll = ((2.0*ll-1.0)*x*pmmp1 - (ll+m-1.0)*pmm)/(ll-m)
        pmm = pmmp1
        pmmp1 = pll
    return pll

def factorial(n):
    return torch.prod(torch.arange(1,n+1), dtype=torch.float32)

def K(l, m):
    norm_const = ((2.0*l+1.0)*factorial(l-m))/((4.0*np.pi)*factorial(l+m))
    return torch.sqrt(norm_const)

def SH(m, l, phi, theta):
    sqrt2 = torch.sqrt(torch.tensor(2.0, dtype=torch.float32))
    if m==0:
        return K(l,0)*P(l,m,torch.cos(theta))
    elif m>0:
        return sqrt2*K(l,m)*torch.cos(m*phi)*P(l,m,torch.cos(theta))
    else:
        return sqrt2*K(l,-m)*torch.sin(-m*phi)*P(l,-m,torch.cos(theta))

In [16]:
def rot_mat(pitch, yaw, roll):
    Ry = torch.tensor([[torch.cos(pitch),0,torch.sin(pitch)],[0,1,0],[-torch.sin(pitch),0,torch.cos(pitch)]], requires_grad = True)
    Rx = torch.tensor([[1,0,0],[0,torch.cos(roll),torch.sin(roll)],[0,-torch.sin(roll),torch.cos(roll)]], requires_grad = True)
    Rz = torch.tensor([[torch.cos(yaw),torch.sin(yaw),0],[-torch.sin(yaw),torch.cos(yaw),0],[0,0,1]], requires_grad = True)
    R = Rz@Ry@Rx
    
    return R

def sh_basis(norm):
#     if torch.is_tensor(n):
#         norm = n.cpu().detach().numpy()
#     else:
#         norm = n
    theta = norm[1] #Polar angle
    phi = norm[0] #Azimuth angle
    sh = torch.zeros((9,), dtype=torch.float32, requires_grad=True)
    count = 0
    for l in torch.arange(3):
        for m in torch.arange(-l,l+1):
#             if m==0:
#                 sh[count]=np.real(sph_harm(m,l,phi,theta))
#             elif m>0:
#                 sh[count]=np.sqrt(2)*np.real(sph_harm(m,l,phi,theta))
#             else:
#                 sh[count]=np.sqrt(2)*np.imag(sph_harm(m,l,phi,theta))
#             count = count+1
                sh[count] = SH(m,l,phi,theta)
                count = count+1
            
    return sh

In [17]:
def barycentric_weights(p, tri_p):
    #http://blackpawn.com/texts/pointinpoly/
    
    a = tri_p[0,:]
    b = tri_p[1,:]
    c = tri_p[2,:]
    
    v0 = c - a
    v1 = b - a
    v2 = p - a
    
    dot00 = np.dot(v0, v0)
    dot01 = np.dot(v0, v1)
    dot02 = np.dot(v0, v2)
    dot11 = np.dot(v1, v1)
    dot12 = np.dot(v1, v2)
    
    denom = dot00*dot11 - dot01*dot01
    
    if denom == 0:
        u = v = 0
    else:
        u = (dot11*dot02-dot01*dot12)/denom
        v = (dot00*dot12-dot01*dot02)/denom
    
#     A = torch.stack((v0,v1)).t()
#     B = v2[:,None]
    
#     X, LU = torch.solve(B,A)
#     u = X[0,0]
#     v = X[1,0]
        
#     weights = torch.tensor([1-u-v, u, v], dtype=torch.float32, requires_grad = True)
    weights = np.array([1-u-v, u, v])
    return weights

def world_to_image(q_world, h, w):
#     temp = np.array([w/2,h/2-h+1,0])
#     q_image = (q_world + temp)*[1,-1,1]
    q_image = q_world.clone()
    
    q_image[:,0] = q_image[:,0] + w/2
    q_image[:,1] = q_image[:,1] + h/2
    q_image[:,1] = h - q_image[:,1] - 1
    
    return q_image

def rasterize(q, q_depth, tri_mesh_data, h, w):
    depth_info = -math.inf*np.ones((h,w))
#     depth_info = {}
    tri_ind_info = -np.ones((h,w))
    bary_wts_info = torch.zeros((h,w,3), requires_grad = True)
    
#     for i in range(h):
#         for j in range(w):
#             depth_info[(i,j)] = -math.inf
#             bary_wts_info[(i,j)] = 0
    q_n = q.data.cpu().numpy()
    q_depth_n = q_depth.data.cpu().numpy()
    for i in range(len(tri_mesh_data)):
        print('Rasterizing triangle: ', i+1)
        tri_ver_ind = tri_mesh_data[i,:]
        
        umin = max(int(np.ceil(np.min(q_n[tri_ver_ind, 0]))), 0) #torch.min(lmks_2d[:,0])
        umax = min(int(np.floor(np.max(q_n[tri_ver_ind, 0]))), w-1) #torch.max(lmks_2d[:,0])
        
        vmin = max(int(np.ceil(np.min(q_n[tri_ver_ind, 1]))), 0) #torch.min(lmks_2d[:,0])
        vmax = min(int(np.floor(np.max(q_n[tri_ver_ind, 1]))), h-1)
        
        if umax<umin or vmax<vmin:
            continue
        else:
            for u in range(umin, umax+1):
                for v in range(vmin, vmax+1):
                    weights = barycentric_weights([u, v], q_n[tri_ver_ind, :])
                    if (weights<0).all():
                        continue
                    else:
                        depth = np.dot(weights, q_depth_n[tri_ver_ind])
                        if depth > depth_info[v,u]:
                            depth_info[v,u] = depth
                            tri_ind_info[v,u] = i
                            bary_wts_info[v,u,:] = torch.tensor(weights, requires_grad=True)
                            
    return tri_ind_info, bary_wts_info

In [18]:
def cart2sph(n):
#     phi = np.arctan2(n[1],n[0]) #arctan(y/x)
#     theta = np.arccos(n[2]) #arccos(z)
    if torch.is_tensor(n):
        norm = n.cpu().detach().numpy()
    else:
        norm = n
    phi = np.arctan2(norm[1],norm[0])
    theta = np.arccos(norm[2])
    return torch.tensor([phi, theta], dtype=torch.float32, requires_grad = True)

def calculate_normal(tri_ind_info, tri_mesh_data, centroid, q, h, w):
    normal_xyz = np.zeros((h, w, 3))
    normal_sph = torch.zeros((h, w, 2), requires_grad = True)
    cent = centroid.data.cpu().numpy()
    for i in range(h):
        for j in range(w):
            tri_ver = q[tri_mesh_data[tri_ind_info[i, j]-1, :],:].data.cpu().numpy()
            a = tri_ver[0,:]
            b = tri_ver[1,:]
            c = tri_ver[2,:]
            normal_xyz[i,j,:] = np.cross(a-b, b-c)/np.linalg.norm(np.cross(a-b, b-c))
            if np.dot(np.mean(tri_ver, 0)-cent, normal_xyz[i,j,:])<0:
                normal_xyz[i,j,:] *= -1
            normal_sph[i,j,:] = cart2sph(normal_xyz[i,j,:])
    return normal_sph

In [19]:
def isPointInTri(point, tri_points):
    ''' Judge whether the point is in the triangle
    Method:
        http://blackpawn.com/texts/pointinpoly/
    Args:
        point: (2,). [u, v] or [x, y] 
        tri_points: (3 vertices, 2 coords). three vertices(2d points) of a triangle. 
    Returns:
        bool: true for in triangle
    '''
    tp = tri_points

    # vectors
    v0 = tp[2,:] - tp[0,:]
    v1 = tp[1,:] - tp[0,:]
    v2 = point - tp[0,:]
#     A = torch.stack((v0,v1)).t()
#     B = v2[:,None]

#     X = torch.inverse(A)@B

#     u = X[0,0]
#     v = X[1,0]
    # dot products
    dot00 = np.dot(v0, v0)
    dot01 = np.dot(v0, v1)
    dot02 = np.dot(v0, v2)
    dot11 = np.dot(v1, v1)
    dot12 = np.dot(v1, v2)

    # barycentric coordinates
    if dot00*dot11 - dot01*dot01 == 0:
        inverDeno = 0
    else:
        inverDeno = 1/(dot00*dot11 - dot01*dot01)

    u = (dot11*dot02 - dot01*dot12)*inverDeno
    v = (dot00*dot12 - dot01*dot02)*inverDeno

    # check if point in triangle
    return (u >= 0) & (v >= 0) & (u + v < 1)

def get_point_weight(point, tri_points):
    ''' Get the weights of the position
    Methods: https://gamedev.stackexchange.com/questions/23743/whats-the-most-efficient-way-to-find-barycentric-coordinates
     -m1.compute the area of the triangles formed by embedding the point P inside the triangle
     -m2.Christer Ericson's book "Real-Time Collision Detection". faster.(used)
    Args:
        point: (2,). [u, v] or [x, y] 
        tri_points: (3 vertices, 2 coords). three vertices(2d points) of a triangle. 
    Returns:
        w0: weight of v0
        w1: weight of v1
        w2: weight of v3
     '''
    tp = tri_points
    # vectors
    v0 = tp[2,:] - tp[0,:]
    v1 = tp[1,:] - tp[0,:]
    v2 = point - tp[0,:]
    
#     A = torch.stack((v0,v1)).t()
#     B = v2[:,None]
    
#     X = torch.inverse(A)@B
#     u = X[0,0]
#     v = X[1,0]
#     dot products
    dot00 = np.dot(v0, v0)
    dot01 = np.dot(v0, v1)
    dot02 = np.dot(v0, v2)
    dot11 = np.dot(v1, v1)
    dot12 = np.dot(v1, v2)

    # barycentric coordinates
    if dot00*dot11 - dot01*dot01 == 0:
        inverDeno = 0
    else:
        inverDeno = 1/(dot00*dot11 - dot01*dot01)

    u = (dot11*dot02 - dot01*dot12)*inverDeno
    v = (dot00*dot12 - dot01*dot02)*inverDeno

   
    w0 = 1 - u - v
    w1 = v
    w2 = u
    return w0,w1,w2

def rasterize_triangles(vertices, triangles, h, w):
    ''' 
    Args:
        vertices: [nver, 3]
        triangles: [ntri, 3]
        h: height
        w: width
    Returns:
        depth_buffer: [h, w] saves the depth, here, the bigger the z, the fronter the point.
        triangle_buffer: [h, w] saves the tri id(-1 for no triangle). 
        barycentric_weight: [h, w, 3] saves corresponding barycentric weight.

    # Each triangle has 3 vertices & Each vertex has 3 coordinates x, y, z.
    # h, w is the size of rendering
    '''
    # initial 
    depth_buffer = torch.zeros([h, w]) - 999999. #+ torch.min(vertices[2,:]) - 999999. # set the initial z to the farest position
    triangle_buffer = torch.zeros([h, w], dtype = torch.int32) - 1  # if tri id = -1, the pixel has no triangle correspondance
    barycentric_weight = torch.zeros([h, w, 3], dtype = torch.float32)  # 
    
#     for i in range(h):
#         for j in range(w):
#             depth_buffer[(i,j)] = -math.inf
#             barycentric_weight[(i,j)] = 0
    ver = vertices.data.cpu().numpy()
    for i in range(triangles.shape[0]):
        print('Rasterzing: ',i+1)
        tri = triangles[i, :] # 3 vertex indices

        # the inner bounding box
        umin = max(int(np.ceil(np.min(ver[tri, 0]))), 0)
        umax = min(int(np.floor(np.max(ver[tri, 0]))), w-1)

        vmin = max(int(np.ceil(np.min(ver[tri, 1]))), 0)
        vmax = min(int(np.floor(np.max(ver[tri, 1]))), h-1)

        if umax<umin or vmax<vmin:
            continue

        for u in range(umin, umax+1):
            for v in range(vmin, vmax+1):
                if not isPointInTri([u, v], ver[tri, :2]): 
                    continue
                w0, w1, w2 = get_point_weight([u, v], ver[tri, :2]) # barycentric weight
                point_depth = w0*vertices[tri[0], 2] + w1*vertices[tri[1], 2] + w2*vertices[tri[2], 2]
                if point_depth > depth_buffer[v, u]:
                    depth_buffer[(v, u)] = point_depth
                    triangle_buffer[v, u] = i
                    barycentric_weight[v, u, :] = torch.tensor([w0, w1, w2], dtype=torch.float32, requires_grad=True)

    return triangle_buffer, barycentric_weight

In [20]:
def render_color_image(q, albedo, tri_mesh_data, gamma, h, w):
    image = torch.zeros((h,w,3), dtype=torch.float32, requires_grad = True)
    alb = torch.zeros((h,w,3), dtype=torch.float32, requires_grad = True)
    centroid = torch.mean(q,0)
    
    st = time()
#     tri_ind_info, bary_wts_info = rasterize(q[:,:2], q[:,2], tri_mesh_data, h, w)
    tri_ind_info, bary_wts_info = rasterize_triangles(q, tri_mesh_data, h, w)
    print('Rasterization Done!- %f seconds' %(time()-st))
    
    n_sph = calculate_normal(tri_ind_info, tri_mesh_data, centroid, q, h, w)
    
    for i in range(h):
        for j in range(w):
            sh_func = sh_basis(n_sph[i,j,:])
            alb[i,j,:] = albedo[tri_mesh_data[tri_ind_info[i, j]-1, :],:].t()@bary_wts_info[i,j,:]
            image[i,j,:] = alb[i,j,:]*(gamma.t()@sh_func.squeeze())
            
    return image

In [21]:
I_in = plt.imread('Dataset/300W-Convert/300W-Original/afw/134212_1.jpg')
I_in=torch.tensor(I_in[160:384,704:928,:],dtype = torch.float32, requires_grad = True)

In [22]:
def compute_jacobian(inputs, output):
    """
    :param inputs: Batch X Size (e.g. Depth X Width X Height)
    :param output: Batch X Classes
    :return: jacobian: Batch X Classes X Size
    """
    assert inputs.requires_grad

    num_classes = output.shape[0]

    jacobian = torch.zeros((num_classes, inputs.shape[0], inputs.shape[1]))
    grad_output = torch.zeros(output.shape)
    if inputs.is_cuda:
        grad_output = grad_output.cuda()
        jacobian = jacobian.cuda()

    for i in range(num_classes):
        zero_gradients(inputs)
        grad_output.zero_()
        grad_output[i] = 1
        output.backward(grad_output, retain_graph=True)
        jacobian[i] = inputs.grad.data

    return jacobian.squeeze()

In [24]:
h=w=224
chi = torch.rand(312,1,requires_grad = True, dtype=torch.float32)
# t = torch.zeros(3,1, requires_grad = True, dtype=torch.float32)
count = 1
chi_prev = chi
while True:
    print("Iteration No: ", count)
    
    al_id = chi_prev[0:100]
    al_exp = chi_prev[100:179]
    al_alb = chi_prev[179:279]
    [s, pitch, yaw, roll] = chi_prev[279:283,0]
    t = chi_prev[283:285]
    r = chi_prev[285:]
    gamma = torch.reshape(r,(3,9)).t()
    
    
    p = p_mu + torch.matmul(A_id,al_id) + torch.matmul(A_exp,al_exp)
    b = b_mu + torch.matmul(A_alb,al_alb)
    vertex = torch.reshape(p, (no_of_ver, 3))
    albedo = torch.reshape(b, (no_of_ver, 3))
    
    if count == 1:
        s = 150/(torch.max(vertex) - torch.min(vertex))
    
    R = rot_mat(pitch, yaw, roll)
    q_world = s*R@vertex.t() #+ t
    q_world[:2,:] = q_world[:2,:] + t
    q_image = world_to_image(q_world.t(), h, w)
    print('Rendering Image...')
    st = time()
#     albe, bary_wts_info = render_color_image(q_image, albedo, tri_mesh_data, gamma, h, w)
    I_rend = render_color_image(q_image, albedo, tri_mesh_data, gamma, h, w)
    print('Rendering Done! - %f seconds' %(time()-st) )
    w_l = 100
    w_r = 5e-5
    #E=torch.norm(I_rend - I_in)**2 + torch.norm(al_id/std_id)**2 + w_r*torch.norm(al_alb/std_alb)**2 + torch.norm(al_exp/std_exp)**2
    E_con_r = torch.tensor([np.sqrt(1/28241)*torch.norm(I_rend - I_in)], requires_grad = True)
    E_lan_r = np.sqrt(w_l/68)*torch.norm(lmks_2d - q_image[lmks_3d_ind[0,:],:2], dim=1)
    E_reg_r = np.sqrt(w_r)*torch.cat((al_id/std_id,al_alb/std_alb,al_exp/std_exp))

    E = torch.cat((E_con_r,E_lan_r,E_reg_r[:,0]))
    J = compute_jacobian(chi_prev,E)
    chi_next = chi_prev - torch.pinverse(J.t()@J)@J.t()@E.unsqueeze(1)
    increment = torch.norm(chi_next - chi_prev)
    obj_func_loss = torch.norm(E)**2
    chi_prev = chi_next
    count=count+1
    print('Increment: ', increment)
    print('Loss: ', obj_func_loss)
    if obj_func_loss<10:
        break

Iteration No:  1
Rendering Image...
Rasterzing:  1
Rasterzing:  2
Rasterzing:  3
Rasterzing:  4
Rasterzing:  5
Rasterzing:  6
Rasterzing:  7
Rasterzing:  8
Rasterzing:  9
Rasterzing:  10
Rasterzing:  11
Rasterzing:  12
Rasterzing:  13
Rasterzing:  14
Rasterzing:  15
Rasterzing:  16
Rasterzing:  17
Rasterzing:  18
Rasterzing:  19
Rasterzing:  20
Rasterzing:  21
Rasterzing:  22
Rasterzing:  23
Rasterzing:  24
Rasterzing:  25
Rasterzing:  26
Rasterzing:  27
Rasterzing:  28
Rasterzing:  29
Rasterzing:  30
Rasterzing:  31
Rasterzing:  32
Rasterzing:  33
Rasterzing:  34
Rasterzing:  35
Rasterzing:  36
Rasterzing:  37
Rasterzing:  38
Rasterzing:  39
Rasterzing:  40
Rasterzing:  41
Rasterzing:  42
Rasterzing:  43
Rasterzing:  44
Rasterzing:  45
Rasterzing:  46
Rasterzing:  47
Rasterzing:  48
Rasterzing:  49
Rasterzing:  50
Rasterzing:  51
Rasterzing:  52
Rasterzing:  53
Rasterzing:  54
Rasterzing:  55
Rasterzing:  56
Rasterzing:  57
Rasterzing:  58
Rasterzing:  59
Rasterzing:  60
Rasterzing:  

Rasterzing:  513
Rasterzing:  514
Rasterzing:  515
Rasterzing:  516
Rasterzing:  517
Rasterzing:  518
Rasterzing:  519
Rasterzing:  520
Rasterzing:  521
Rasterzing:  522
Rasterzing:  523
Rasterzing:  524
Rasterzing:  525
Rasterzing:  526
Rasterzing:  527
Rasterzing:  528
Rasterzing:  529
Rasterzing:  530
Rasterzing:  531
Rasterzing:  532
Rasterzing:  533
Rasterzing:  534
Rasterzing:  535
Rasterzing:  536
Rasterzing:  537
Rasterzing:  538
Rasterzing:  539
Rasterzing:  540
Rasterzing:  541
Rasterzing:  542
Rasterzing:  543
Rasterzing:  544
Rasterzing:  545
Rasterzing:  546
Rasterzing:  547
Rasterzing:  548
Rasterzing:  549
Rasterzing:  550
Rasterzing:  551
Rasterzing:  552
Rasterzing:  553
Rasterzing:  554
Rasterzing:  555
Rasterzing:  556
Rasterzing:  557
Rasterzing:  558
Rasterzing:  559
Rasterzing:  560
Rasterzing:  561
Rasterzing:  562
Rasterzing:  563
Rasterzing:  564
Rasterzing:  565
Rasterzing:  566
Rasterzing:  567
Rasterzing:  568
Rasterzing:  569
Rasterzing:  570
Rasterzing:  5

Rasterzing:  1025
Rasterzing:  1026
Rasterzing:  1027
Rasterzing:  1028
Rasterzing:  1029
Rasterzing:  1030
Rasterzing:  1031
Rasterzing:  1032
Rasterzing:  1033
Rasterzing:  1034
Rasterzing:  1035
Rasterzing:  1036
Rasterzing:  1037
Rasterzing:  1038
Rasterzing:  1039
Rasterzing:  1040
Rasterzing:  1041
Rasterzing:  1042
Rasterzing:  1043
Rasterzing:  1044
Rasterzing:  1045
Rasterzing:  1046
Rasterzing:  1047
Rasterzing:  1048
Rasterzing:  1049
Rasterzing:  1050
Rasterzing:  1051
Rasterzing:  1052
Rasterzing:  1053
Rasterzing:  1054
Rasterzing:  1055
Rasterzing:  1056
Rasterzing:  1057
Rasterzing:  1058
Rasterzing:  1059
Rasterzing:  1060
Rasterzing:  1061
Rasterzing:  1062
Rasterzing:  1063
Rasterzing:  1064
Rasterzing:  1065
Rasterzing:  1066
Rasterzing:  1067
Rasterzing:  1068
Rasterzing:  1069
Rasterzing:  1070
Rasterzing:  1071
Rasterzing:  1072
Rasterzing:  1073
Rasterzing:  1074
Rasterzing:  1075
Rasterzing:  1076
Rasterzing:  1077
Rasterzing:  1078
Rasterzing:  1079
Rasterzing

Rasterzing:  1537
Rasterzing:  1538
Rasterzing:  1539
Rasterzing:  1540
Rasterzing:  1541
Rasterzing:  1542
Rasterzing:  1543
Rasterzing:  1544
Rasterzing:  1545
Rasterzing:  1546
Rasterzing:  1547
Rasterzing:  1548
Rasterzing:  1549
Rasterzing:  1550
Rasterzing:  1551
Rasterzing:  1552
Rasterzing:  1553
Rasterzing:  1554
Rasterzing:  1555
Rasterzing:  1556
Rasterzing:  1557
Rasterzing:  1558
Rasterzing:  1559
Rasterzing:  1560
Rasterzing:  1561
Rasterzing:  1562
Rasterzing:  1563
Rasterzing:  1564
Rasterzing:  1565
Rasterzing:  1566
Rasterzing:  1567
Rasterzing:  1568
Rasterzing:  1569
Rasterzing:  1570
Rasterzing:  1571
Rasterzing:  1572
Rasterzing:  1573
Rasterzing:  1574
Rasterzing:  1575
Rasterzing:  1576
Rasterzing:  1577
Rasterzing:  1578
Rasterzing:  1579
Rasterzing:  1580
Rasterzing:  1581
Rasterzing:  1582
Rasterzing:  1583
Rasterzing:  1584
Rasterzing:  1585
Rasterzing:  1586
Rasterzing:  1587
Rasterzing:  1588
Rasterzing:  1589
Rasterzing:  1590
Rasterzing:  1591
Rasterzing

Rasterzing:  2049
Rasterzing:  2050
Rasterzing:  2051
Rasterzing:  2052
Rasterzing:  2053
Rasterzing:  2054
Rasterzing:  2055
Rasterzing:  2056
Rasterzing:  2057
Rasterzing:  2058
Rasterzing:  2059
Rasterzing:  2060
Rasterzing:  2061
Rasterzing:  2062
Rasterzing:  2063
Rasterzing:  2064
Rasterzing:  2065
Rasterzing:  2066
Rasterzing:  2067
Rasterzing:  2068
Rasterzing:  2069
Rasterzing:  2070
Rasterzing:  2071
Rasterzing:  2072
Rasterzing:  2073
Rasterzing:  2074
Rasterzing:  2075
Rasterzing:  2076
Rasterzing:  2077
Rasterzing:  2078
Rasterzing:  2079
Rasterzing:  2080
Rasterzing:  2081
Rasterzing:  2082
Rasterzing:  2083
Rasterzing:  2084
Rasterzing:  2085
Rasterzing:  2086
Rasterzing:  2087
Rasterzing:  2088
Rasterzing:  2089
Rasterzing:  2090
Rasterzing:  2091
Rasterzing:  2092
Rasterzing:  2093
Rasterzing:  2094
Rasterzing:  2095
Rasterzing:  2096
Rasterzing:  2097
Rasterzing:  2098
Rasterzing:  2099
Rasterzing:  2100
Rasterzing:  2101
Rasterzing:  2102
Rasterzing:  2103
Rasterzing

...

Rasterzing:  3577
Rasterzing:  3578
Rasterzing:  3579
Rasterzing:  3580
Rasterzing:  3581
Rasterzing:  3582
Rasterzing:  3583
Rasterzing:  3584
Rasterzing:  3585
Rasterzing:  3586
Rasterzing:  3587
Rasterzing:  3588
Rasterzing:  3589
Rasterzing:  3590
Rasterzing:  3591
Rasterzing:  3592
Rasterzing:  3593
Rasterzing:  3594
Rasterzing:  3595
Rasterzing:  3596
Rasterzing:  3597
Rasterzing:  3598
Rasterzing:  3599
Rasterzing:  3600
Rasterzing:  3601
Rasterzing:  3602
Rasterzing:  3603
Rasterzing:  3604
Rasterzing:  3605
Rasterzing:  3606
Rasterzing:  3607
Rasterzing:  3608
Rasterzing:  3609
Rasterzing:  3610
Rasterzing:  3611
Rasterzing:  3612
Rasterzing:  3613
Rasterzing:  3614
Rasterzing:  3615
Rasterzing:  3616
Rasterzing:  3617
Rasterzing:  3618
Rasterzing:  3619
Rasterzing:  3620
Rasterzing:  3621
Rasterzing:  3622
Rasterzing:  3623
Rasterzing:  3624
Rasterzing:  3625
Rasterzing:  3626
Rasterzing:  3627
Rasterzing:  3628
Rasterzing:  3629
Rasterzing:  3630
Rasterzing:  3631
Rasterzing

Rasterzing:  4097
Rasterzing:  4098
Rasterzing:  4099
Rasterzing:  4100
Rasterzing:  4101
Rasterzing:  4102
Rasterzing:  4103
Rasterzing:  4104
Rasterzing:  4105
Rasterzing:  4106
Rasterzing:  4107
Rasterzing:  4108
Rasterzing:  4109
Rasterzing:  4110
Rasterzing:  4111
Rasterzing:  4112
Rasterzing:  4113
Rasterzing:  4114
Rasterzing:  4115
Rasterzing:  4116
Rasterzing:  4117
Rasterzing:  4118
Rasterzing:  4119
Rasterzing:  4120
Rasterzing:  4121
Rasterzing:  4122
Rasterzing:  4123
Rasterzing:  4124
Rasterzing:  4125
Rasterzing:  4126
Rasterzing:  4127
Rasterzing:  4128
Rasterzing:  4129
Rasterzing:  4130
Rasterzing:  4131
Rasterzing:  4132
Rasterzing:  4133
Rasterzing:  4134
Rasterzing:  4135
Rasterzing:  4136
Rasterzing:  4137
Rasterzing:  4138
Rasterzing:  4139
Rasterzing:  4140
Rasterzing:  4141
Rasterzing:  4142
Rasterzing:  4143
Rasterzing:  4144
Rasterzing:  4145
Rasterzing:  4146
Rasterzing:  4147
Rasterzing:  4148
Rasterzing:  4149
Rasterzing:  4150
Rasterzing:  4151
Rasterzing

Rasterzing:  4608
Rasterzing:  4609
Rasterzing:  4610
Rasterzing:  4611
Rasterzing:  4612
Rasterzing:  4613
Rasterzing:  4614
Rasterzing:  4615
Rasterzing:  4616
Rasterzing:  4617
Rasterzing:  4618
Rasterzing:  4619
Rasterzing:  4620
Rasterzing:  4621
Rasterzing:  4622
Rasterzing:  4623
Rasterzing:  4624
Rasterzing:  4625
Rasterzing:  4626
Rasterzing:  4627
Rasterzing:  4628
Rasterzing:  4629
Rasterzing:  4630
Rasterzing:  4631
Rasterzing:  4632
Rasterzing:  4633
Rasterzing:  4634
Rasterzing:  4635
Rasterzing:  4636
Rasterzing:  4637
Rasterzing:  4638
Rasterzing:  4639
Rasterzing:  4640
Rasterzing:  4641
Rasterzing:  4642
Rasterzing:  4643
Rasterzing:  4644
Rasterzing:  4645
Rasterzing:  4646
Rasterzing:  4647
Rasterzing:  4648
Rasterzing:  4649
Rasterzing:  4650
Rasterzing:  4651
Rasterzing:  4652
Rasterzing:  4653
Rasterzing:  4654
Rasterzing:  4655
Rasterzing:  4656
Rasterzing:  4657
Rasterzing:  4658
Rasterzing:  4659
Rasterzing:  4660
Rasterzing:  4661
Rasterzing:  4662
Rasterzing

Rasterzing:  5120
Rasterzing:  5121
Rasterzing:  5122
Rasterzing:  5123
Rasterzing:  5124
Rasterzing:  5125
Rasterzing:  5126
Rasterzing:  5127
Rasterzing:  5128
Rasterzing:  5129
Rasterzing:  5130
Rasterzing:  5131
Rasterzing:  5132
Rasterzing:  5133
Rasterzing:  5134
Rasterzing:  5135
Rasterzing:  5136
Rasterzing:  5137
Rasterzing:  5138
Rasterzing:  5139
Rasterzing:  5140
Rasterzing:  5141
Rasterzing:  5142
Rasterzing:  5143
Rasterzing:  5144
Rasterzing:  5145
Rasterzing:  5146
Rasterzing:  5147
Rasterzing:  5148
Rasterzing:  5149
Rasterzing:  5150
Rasterzing:  5151
Rasterzing:  5152
Rasterzing:  5153
Rasterzing:  5154
Rasterzing:  5155
Rasterzing:  5156
Rasterzing:  5157
Rasterzing:  5158
Rasterzing:  5159
Rasterzing:  5160
Rasterzing:  5161
Rasterzing:  5162
Rasterzing:  5163
Rasterzing:  5164
Rasterzing:  5165
Rasterzing:  5166
Rasterzing:  5167
Rasterzing:  5168
Rasterzing:  5169
Rasterzing:  5170
Rasterzing:  5171
Rasterzing:  5172
Rasterzing:  5173
Rasterzing:  5174
Rasterzing

KeyboardInterrupt: 

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(q_world[0,:].cpu().detach(), q_world[1,:].cpu().detach(), q_world[2,:].cpu().detach())

In [ ]:
ax.view_init(-45,45)
fig

In [1]:
E_con_r

NameError: name 'E_con_r' is not defined